# Vlákna (Threading) v Pythonu

Python má pro vlákna modul `threading`. Je důležité vědět, že v Pythonu (implementace CPython) existuje **GIL (Global Interpreter Lock)**. To znamená, že **Python kód** může běžet v jednu chvíli vždy jen na jednom jádře procesoru.

Proto jsou vlákna v Pythonu skvělá na **čekání** (stahování dat, disk, databáze), ale špatná na **počítání**.

## 1. ThreadPoolExecutor (Moderní přístup)
Místo manuálního vytváření vláken (`t = threading.Thread...`) je v moderním Pythonu lepší použít `concurrent.futures`. Je to obdoba "Poolu" vláken.

In [ ]:
import concurrent.futures
import time

def usni_na(sekundy):
    print(f"Jdu spát na {sekundy}s...")
    time.sleep(sekundy)
    return f"Probuzen po {sekundy}s"

# Vytvoříme pool 3 dělníků (vláken)
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # Zadáme jim práci (časy spánku)
    casy = [2, 1, 3]
    # map spustí funkci pro každý prvek v seznamu, ale paralelně
    vysledky = executor.map(usni_na, casy)

    for vysledek in vysledky:
        print(vysledek)

print("Vše hotovo.")

## 2. Producent - Konzument (Fronta)
Toto je klasický návrhový vzor. Jedno vlákno data vyrábí, druhé je zpracovává.
V Pythonu je `queue.Queue` **thread-safe**, takže nemusíme řešit zámky (Lock) při vkládání a vybírání.

In [ ]:
import threading
import queue
import time
import random

def producent(q, id_producenta):
    for i in range(5):
        polozka = f"Data {id_producenta}-{i}"
        time.sleep(random.uniform(0.1, 0.5))
        q.put(polozka)
        print(f"[Producent {id_producenta}] Vyrobil: {polozka}")

def konzument(q, id_konzumenta):
    while True:
        # Získáme položku (blokuje, pokud je fronta prázdná)
        polozka = q.get()
        if polozka is None: # Signál k ukončení
            break
        
        print(f"  -> [Konzument {id_konzumenta}] Zpracoval: {polozka}")
        time.sleep(0.5)
        q.task_done() # Řekneme frontě, že úkol je hotový

# Fronta práce
prace = queue.Queue()

# Spustíme konzumenta (dělníka)
t_konzument = threading.Thread(target=konzument, args=(prace, 1))
t_konzument.start()

# Spustíme producenta
t_producent = threading.Thread(target=producent, args=(prace, 1))
t_producent.start()

# Počkáme na producenta
t_producent.join()

# Pošleme signál k ukončení konzumentovi
prace.put(None)
t_konzument.join()

print("Hotovo.")

## Cvičení
1. Vytvořte vlákno, které bude každou sekundu vypisovat aktuální čas.
2. Hlavní program mezitím čeká na `input()` od uživatele.
3. Jakmile uživatel něco napíše, vlákno s časem se ukončí (použijte globální proměnnou `stop_signal` nebo `threading.Event`).

In [ ]:
# Zde napište řešení...